# Unit 5: Statistical Analysis
------------------------------

Understanding statistical analysis helps the research engineer understand significance of an experimental result and the manufacturing engineer understand process variability. In this section, we will introduce several basic statistical concepts using the `scipy.stats` Python package. This section will assume a basic understanding of statistical terms such as mean, median, mode, variance and standard deviation.

**After completing this unit, you should be able to:**

- Calculate and plot the histogram for a dataset
- Generate random samples from a probability distribution
- Understand the Central Limit Theorem, and implications on variance in experimental data
- Create an x-bar R run chart
- Conduct a t-test and calculate a confidence interval for a sample mean

## 5.1. Calculating and plotting a histogram

The histogram is a common visualization taught in introductory statistics classes to summarize large sets of measurements. Instead of looking at each individual data point, we define a set of *bins* that are relevent for our dataset. For example, to study ages of individuals in a population, we might consider bins with a width of 10 years (0-9 years, 10-19 years, 20-29 years, ...). The number of measurements falling within each bin is counted. The resulting counts are typically visualized as a bar graph with no space in between the bars. Sometimes, each bin count is scaled so that the total area under the curve is equal to 1. Scaling in this way can make it easier to compare the shape of two sets of data that may have different lengths.

### 5.1.1. Plotting the histogram using `matplotlib`

In the example below, we load a set of tensile modulus measurements for polymer films into an `np.ndarray`. Try adding code to the cell below to determine how many values are in this data set. The `matplotlib` function [`Axes.hist()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.hist.html) will generate the histogram of the supplied data. By default, the data will be split into 10 bins, evenly spaced between the minimum and maximum values of the data. Starting with these default settings we can quickly identify that this dataset appears to be bi-modal, which *may* indicate that it represents two different polymer films (or different manufacturing lots of film) that are statistically different. We will further analyze this distribution later in the unit to determine if there is evidence (beyond visual appearance) to support this assumption.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# load the modulus dataset into memory as a 1d array
modulus = np.loadtxt('../../data/bopet_modulus-MPa.csv')

# create the plot
fig, ax = plt.subplots() 

# add the histogram to the axis
ax.hist(modulus)

# update the axis labels
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

### 5.1.2. Adjusting the histogram bins

The `bins` parameter of the `Axes.hist()` function is used to set a number of bins (as an `integer`) or the bin edges (as a `list` or `np.ndarray`). Study the histogram below, with the `bins` parameter set to 50. Modify the bin count and observe how the histogram changes. While it might seem like more bins will always improve resolution, there is a point at which you make the bins too narrow and you lose sight of the distribution. If many of the bins contain a single measured value, there are too many bins. This examples also demonstrates the `density` parameter, which can be set to `True` to scale the histogram so that the sum of the areas of the bars is equal to 1.

$$\sum \left( \text{bin width} \right) \times \left( \text{bar height} \right) = 1$$

In [ ]:
fig, ax = plt.subplots()

# modify the bins parameter to change the bin count
ax.hist(modulus, bins=50, density=True)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Density')

When setting the bin count, the width of the bins will scale based on the range of the data set. This isn't usually a problem with a single set of data but, when plotting multiple histograms or comparing the histograms from different points in time, we may want to enforce greater consistency in how the bins are created. 

The desired (left) bin edges in place of the bin count. Recall the `np.arange()` function from Unit 2, and feel free to modify the code if needed to print out the bin edge array that has been created. Here, we create bins with a width of 100 MPa, beginning at 4200 MPa and ending with 5800 MPa. Creating bins using a round number like this can make it easier to read data from the plot.

In [ ]:
# array of bin edges to supply to the histogram
bin_edges = np.arange(4200, 5801, 100)

# plot the histogram using the provided bin_edges array
fig, ax = plt.subplots() 

ax.hist(modulus, bins=bin_edges)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

The bin edge array forms a series of half-open intervals for the bins: [4300,4400), [4400,4500), ... However, the final bin will *include* the last value: [5700,5800]. When specifying bin edges with an array, it is important to know that any values outside of the bin range will be clipped from the data. The same modulus data is used for the histogram below, but the `bin_edges` array does not capture the full range of the data. There is no warning that this occurs, so be careful when coding the bin ranges.

In [ ]:
# array of bin edges to supply to the histogram, this does not capture all of the data
bin_edges = np.arange(4600, 5001, 100)

# plot the histogram using the provided bin_edges array
fig, ax = plt.subplots() 

ax.hist(modulus, bins=bin_edges)

ax.set_xticks(bin_edges)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

To get the best of both worlds, you can write code to calculate the `bin_edges` array based on your actual data set. In the code below, the functions `np.floor()` (round down to the nearest integer) and `np.ceil()` (round up to the nearest integer) are used to determine the proper bin edges to use. The resulting `bin_edges` array could be passed to the `Axes.hist()` function to set the bin edges, and would be always capture the entire range of values.

In [ ]:
# set the bin width
bin_width = 100

# calculate the minimum bin edge, rounded down to the nearest bin width
bin_min = bin_width * np.floor(modulus.min() / bin_width)

# calculate the minimum bin edge, rounded up to the nearest bin width
bin_max = bin_width * (np.ceil(modulus.max() / bin_width) + 1)

# use the calculated values to create the bin_edges array
bin_edges = np.arange(bin_min, bin_max, bin_width)
bin_edges

There are also different automated approaches for the creation of bins, which are describe in the [`numpy` documentation](https://numpy.org/doc/stable/reference/generated/numpy.histogram_bin_edges.html#numpy.histogram_bin_edges). To use one of these approaches, provide the algorithm name as the value of the `bins` parameter as shown below.

In [ ]:
# plot the histogram using the sturges method of setting bin count
fig, ax = plt.subplots() 

# use the sturges method to create the bin spacing
ax.hist(modulus, bins='sturges')
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

### 5.1.3. Using the numerical histogram values

Sometimes, it is helpful to save the numerical results of the histogram operation. For example, we could apply the `ndarray.cumsum()` function to calculate and plot the cumulative sum of the bin counts. The `Axes.hist()` function returns a `tuple` of three values:

1. An array of bin counts. These will be normalized if `density=True`.
2. The array of bin edges, generated by any of the methods described above.
3. An object containing the bars generated on the axis by `matplotlib`.

We will focus on the first return value -- the array of counts by bin. Modify the `bins` parameter and see how this changes the `counts` and `bins` arrays.

In [ ]:
# plot the histogram using the pre-defined bin edges from previous cells
fig, ax = plt.subplots() 

counts, bins, bars = ax.hist(modulus, bins=bin_edges)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

# display the counts and bins arrays as a tuple
counts, bins

### 5.1.4. Adding the cumulative sum to the histogram

Using the `counts` array, we can calculate and plot the cumulative distribution. In this example the cumulative distribution is plotted as a line, and the points are centered within the bin range. Review the code that creates the x-coordinate. This averages between the left and right edge of each bin, by stacking up a slice of the array from index 0 to $n-1$ and a second slice from index 1 to $n$. 

$$\begin{bmatrix}
x_0 & x_1 & x_2 & ... & x_{n-1} \\
x_1 & x_2 & x_3 & ... & x_n \\
\end{bmatrix}$$

The average of these two arrays is calculated by column to get the mid-point of the bin.

In [ ]:
# plot the histogram using the pre-defined bin edges from previous cells
fig, ax = plt.subplots() 

counts, bins, bars = ax.hist(modulus, bins=bin_edges)

ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

# calculate the cumulative sum for the y-value
cum_counts = counts.cumsum()

# calculate the center point of each bin for the x-coordinate
# this is calculated by stacking the 0...n-1 bin edges (left edges) with
# the array of 1...n bin edges (right edges) and computing 
# the mean by column (axis=0)
cum_x = np.vstack((bins[:-1], bins[1:])).mean(axis=0)

# plot the cumulative sum as a line
ax.plot(cum_x, cum_counts)

Because the cumulative sum is obviously much larger than the count value in any one bin, the plot above isn't very effective. The histogram is now compressed to the bottom of the plot and the cumulative sum is overwhelmingly large. One approach might be to use separate left and right y-scales, as you might do in a spreadsheet graph. The function `Axes.twinx()` creates a duplicate of the axis, overlaid with the original axis and sharing x-coordinates. The y-axis scale can be different, and the y-ticks will be shown (by default) on the right side of the graph. The second axis is captured as a new variable name and used to plot the cumulative sum.

In [ ]:
# plot the histogram using the pre-defined bin edges from previous cells
fig, ax = plt.subplots() 

counts, bins, bars = ax.hist(modulus, bins=bin_edges)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

# create a duplicate axis, overlaid on the first, sharing the same x
ax2 = ax.twinx()

# plot the cumulative sum on axis 2 as a line
ax2.plot(cum_x, cum_counts)

This is a terrible plot because the default number of ticks is not the same between the two y scales. As a result, there are multiple sets of gridlines that are both confusing and aesthetically displeasing. When using twinned axes in `matplotlib`, it is usually necessary to customize the y limits and tick marks to align the two scales. In the example below, the scales are modified to have a 1:5 ratio between the left and right scales. This results in a plot that is relatively easy to read and understand, where the grid lines are useful for both the left and right scales. The `alpha` parameter is also used to fade the data into the background, which could be used to highlight the cumulative distribution. 

In [ ]:
# plot the histogram using the pre-defined bin edges from previous cells
fig, ax = plt.subplots() 

counts, bins, bars = ax.hist(modulus, bins=bin_edges, alpha=0.4)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Count')

# fix the left y-axis scale
ax.set_ylim((0, 13))
ax.set_yticks(np.arange(0, 13, 2))

# create a duplicate axis and plot the cumulative distribution
ax2 = ax.twinx()
ax2.plot(cum_x, cum_counts)
ax2.set_ylabel('Cumulative Sum')

# fix the right y-axis scale
ax2.set_ylim((0, 65))
ax2.set_yticks(np.arange(0, 61, 10))

# turn off the gridlines on the second axis so that they don't overlay
# the histogram plotted on ax
ax2.grid(False)


## 5.2. The normal probability distribution

In the Unit 1 problems, we created a custom function for the Gaussian (normal) continuous probability density function.

$$f(x)=\frac{1}{\sigma \sqrt{2\pi}}e^{-\frac{1}{2}\frac{\left(x-\mu\right)^2}{\sigma^2}}$$

While this method works, it's more efficient to assume that someone else has already created a module to address common needs like this. The `scipy` project includes a package for statistical analysis called [`scipy.stats`](https://docs.scipy.org/doc/scipy/tutorial/stats.html), which includes many common probability distributions. In this example, we import the package using the line `from scipy import stats`, which imports only the `stats` package from the broader `scipy` project. In the subsequent code, we will preference any calls from this package with `stats.`.

### 5.2.1. Overlaying the probability density function

The `scipy.stats` package contains classes for many common probability distributions, and can be used to calculate values of the a normal probability distribution function for a given mean and standard deviation. The normal distribution is referenced by `stats.norm`. In this package, the parameter `loc` (location) is the distribution mean and `scale` is the standard deviation. To calculate the standard deviation, an additional parameter `ddof=1` is set to indicate that this is *sample* standard deviation (the *degrees of freedom* is $n-1$). The `stats.norm.pdf()` function calculates the value of the *probability density function* at each point in the provided `x` array. We can see by inspection that this is not a good fit, based on the bimodal nature of this dataset.

In [ ]:
from scipy import stats

# plot the histogram using the pre-defined bin edges from previous cells
fig, ax = plt.subplots() 

counts, bins, bars = ax.hist(modulus, bins=bin_edges, density=True, alpha=0.5)
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Frequency')

# create a set of 100 x values at which to evalute the normal pdf
# the space is created 1 stdev above and below the range of the data
x = np.linspace(modulus.min()-modulus.std(ddof=1), modulus.max()+modulus.std(ddof=1), 100)

# calculate the value of the probability density function for each value in x
norm_pdf = stats.norm.pdf(x, loc=modulus.mean(), scale=modulus.std(ddof=1))

# overlay the probability density function in the plot
ax.plot(x, norm_pdf)


### 5.2.2. Cumulative probability distributions

In addition to the probability density function, `stats.norm.pdf(x)`, `scipy.stats` provides other useful functions for working with these probability distributions. For instance, for any continuous probability density function, we can define the cumulative density function as

$$\text{cdf}(x)=\int\limits_{-\infty}^{x}\text{pdf}(x)dx$$

As you can see, this is the cumulative probability of observing any value less than $x$. As $x \rightarrow \infty$, $\text{cdf}(x) \rightarrow 1$; the area under the entire curve is 1. The value of the cdf can be calculated by using the function `stats.norm.cdf(x)`.

In [ ]:
# array of x values for plotting
x = np.linspace(7, 13, 100)

# evaluate the pdf, cdf functions at x
norm_pdf = stats.norm.pdf(x, loc=10, scale=0.6)
norm_cdf = stats.norm.cdf(x, loc=10, scale=0.6)

# plot the pdf, cdf functions
fig, ax = plt.subplots()

ax.plot(x, norm_pdf, label='pdf')
ax.plot(x, norm_cdf, label='cdf')

ax.set_ylabel('Density')
ax.legend()


Alternatively, it may be required to calculate the cumulative probability of a value greater than $x$, which uses the *survival function*, `stats.norm.sf(x)`. Because the total area of the pdf function is equal to 1 it can be shown that

$$\text{sf}(x)=\int\limits_{x}^{\infty}\text{pdf}(x)dx=1-\int\limits_{-\infty}^{x}\text{pdf}(x)dx=1-\text{cdf}(x)$$

In [ ]:
# evaluate the pdf, sf functions at x (using the prior x values)
norm_pdf = stats.norm.pdf(x, loc=10, scale=0.6)
norm_sf = stats.norm.sf(x, loc=10, scale=0.6)

# plot the pdf, sf functions
fig, ax = plt.subplots()

ax.plot(x, norm_pdf, label='pdf')
ax.plot(x, norm_sf, label='sf')

ax.set_ylabel('Density')
ax.legend()

The inverse of the `cdf` and `isf` functions are also available in `scipy.stats`. These can be used to answer questions like, "at what value of $x$ is the $\text{cdf}(x)$<0.2?" In a statistics class, this type of question would typically be answered by consulting a table of values for the normal distribution in the appendix of a textbook, but we can calculate this answer directly using Python.

| Function | Inverse | Description |
|----------|---------|-------------|
| `cdf(x)` | `ppf(q)` | *percentage point function*: at what *x'* is $P(x<x')=p$ |
| `sf(x)` | `isf(q)` | *inverse survival function*: at what *x'* is $P(x>x')=p$ |

Because $\text{cdf}=1-\text{sf}$, it can be shown that $\text{ppf}(q)=\text{isf}(1-q)$.

This example calculates the measurement value corresponding to $\text{cdf}(x)$<0.2, and introduces the [`axis.fill_between()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.fill_between.html) function to shade the area under the curve.

In [ ]:
# plot the pdf, cdf functions using previously calculated values
fig, ax = plt.subplots()

ax.plot(x, norm_pdf, label='pdf')

# the zorder parameter is used to make sure that the cdf is displayed above the fill
ax.plot(x, norm_cdf, label='cdf', zorder=10)

ax.set_ylabel('Density')
ax.legend()

# calculate the x value where cdf(x)=0.2
x_20 = stats.norm.ppf(0.2, loc=10, scale=0.6)

# plot a vertical line at the x_20 value
ax.axvline(x_20, ls='--')

# fill the area of the pdf up to the x_20 value
# use array filters to select the proper points to plot
# use zorder to ensure fill is below cdf
ax.fill_between(x[x<x_20], norm_pdf[x<x_20], alpha=0.4, zorder=1)

### 5.2.3. Sampling random values from the probability distribution

For some simulations, it may be helpful to generate random values from the normal distribution. Recall that in Unit 2, we created random values from a flat probability distribution using `numpy.random.random()`. In `scipy.stats`, the probability distributions include the function `rvs()` (random values). This function takes a parameter named `size` which is the length of the array that will be generated. For the normal distribution, this will be called by the code `stats.norm.rvs()`.

In [ ]:
# generate 200 points from a normal distribution with a mean of 10 
# and a standard deviation of 0.6
norm_random = stats.norm.rvs(size=200, loc=10, scale=0.6)

# plot the histogram of the random values
fig, ax = plt.subplots()
ax.hist(norm_random, density=True)

# plot the probability density function, which was calculated in a previous cell
ax.plot(x, norm_pdf, label='pdf')

ax.set_ylabel('Density')

## 5.3. Constructing control charts

### 5.3.1. Plotting a run chart of the measurements

Let's return to the previous example of polymer modulus measurements, which seemed to exhibit a bimodal distribution. One initial step in this analysis may be to plot out the measurements in time-series order, to see if there is any noticeable trend. Below, we very simply plot the individual values of the modulus, using `numpy.arange()` to create a set of index values for the x coordinate.

In [ ]:
# create an x-coordinate for each point in the dataset
x = np.arange(len(modulus))

# plot modulus values in the order the measurements were made
fig, ax = plt.subplots()

# the Axes.plot() function can plot the line and markers at teh same time
ax.plot(x, modulus, marker='x')

ax.set_xlabel('Index of Data Point')
ax.set_ylabel('Modulus (MPa)')
ax.set_title('Individual Values')

This is interesting, because it looks like the first 30 points have a different range than the last 30 points. This simple run chart, containing just the individual values, exhibits considerable variation. A common technique in statistical process control (SPC) is the $\bar{x}$ ("x-bar") chart, which plots averages of a fixed number of replicate points. The use of control charts such as these is common for industrial engineers and practicioners of the "$6\sigma$" methodology. Engineering courses such as ISyE 512 dive into the detailed theory behind different forms of control charts and potential pitfalls.

The average of consecutive values in an array can be computed by reshaping the 1d array to a 2d matrix which contains a number of columns representing the count of the values that we would like to average together. Look at the results to the following code to check your understanding of what's happening with this step. These sample means can then be plotted as a run chart. Variability is decreased by averaging multiple values together. 

Starting matrix:

$$\begin{bmatrix}
y_0 & y_1 & y_2 & y_3 & y_4 & y_5 & ... & y_n
\end{bmatrix}$$

Reshaped matrix (to average $n=3$ points):

$$\begin{bmatrix}
y_0 & y_1 & y_2 \\
y_3 & y_4 & y_5 \\
... & ... & y_n
\end{bmatrix}$$

The `axis=1` parameter is set in the `mean()` function to average across the rows.

In [ ]:
# reshape the array into a 2d matrix with 3 columns
# -1 rows means that the number of rows will be automatically determined
modulus_grouped = modulus.reshape((-1, 3))

# create a new 1d array by averaging across the columns (axis=1)
x_bar = modulus_grouped.mean(axis=1)

# plot the sample mean values
fig, ax = plt.subplots()

ax.plot(np.arange(len(x_bar)), x_bar, marker='x')

ax.set_xticks(np.arange(len(x_bar)))

ax.set_xlabel('Index of Data Point')
ax.set_ylabel('Modulus (MPa)')
ax.set_title(r'$\bar{x}$ Chart')

# display the first 9 measurements, groupings, and the mean values to check understanding
modulus[:9], modulus_grouped[:3, :], x_bar[:3]

### 5.3.2 Central Limit Theorem

Consider a set of laboratory measurements. What do we do if there is a high degree of variability in the data? Intuitively we might take multiple measurements, and average them together, to reduce noise in the data. This result, that the variability decreases as we average measurements, is grounded in the Central Limit Theorem.

The [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) describes what happens if we randomly draw $n$ samples from population with a mean, $\mu$, and standard deviation, $\sigma$. We can average the samples to get the variable $\bar{x}_n$. The standard deviation of the averages will decrease as $n$ increases:

$$\sigma_{\bar{x}}=\frac{\sigma}{\sqrt{n}}$$

So, as more samples are included in the average, variability is reduced and the power of our measurement is improved. An illustration of the effect of averaging is shown below. A set of 1200 random values is sampled from the standard normal distribution ($\mu=0$, $\sigma=1$). Then, the values are reshaped and averaged using the method described above with increasing values of $n$. The histogram of the *average* values is plotted. As $n$ increases, the histogram gets progressively narrower.

In [ ]:
# create a random sample of 1200 points
total_vals = 1200
norm_random = stats.norm().rvs(size=total_vals)

# plot the histogram, with the normal pdf
fig, ax = plt.subplots()
ax.set_xlim((-3.5, 3.5))

x = np.linspace(-3.5, 3.5, 100)

# number of measurements to average together
n_to_average = [1, 5, 10, 50]

# loop through increasing values of n, and plot the distribution
for n in n_to_average:

    # reshape the 1d array into a 2d matrix (number of cols = n)
    norm_reshape = norm_random.reshape((int(total_vals/n), n))

    # take the mean across the replicate measurements in each row
    norm_means = norm_reshape.mean(axis=1)

    # add a histogram of the averaged values to the axis
    counts, bins, patches = ax.hist(norm_means, density=True, alpha=0.5, 
                                    label=f'$\\bar{{x}}_{{{n}}}$')
    
    # get the color to match the color of the normal pdf to the histogram
    color = patches[0].get_facecolor()

    # add the standard normal pdf to the axis
    ax.plot(x, stats.norm(loc=0, scale=1/np.sqrt(n)).pdf(x), c=color, alpha=1)

ax.legend()

### 5.3.3. Setting control limits for the $\bar{x}$ chart

Now that we understand how averaging values together will affect the variance of our data, we can create *control limits* to bracket the expected values for the measured parameter. Often, control limits are defined in terms of the range in standard deviations, e.g. "$3\sigma$" or "$6\sigma$". The choice of range will determine what types of error are more likely. Narrow limits will increase the likelihood that a point is labeled *out of control* when it is actually representative of the population. A typical value for process monitoring may be a $3\sigma$ limit, where the control limits are given by $\mu \pm 1.5\sigma$.

Wider limits of $6\sigma$ are used for the example below. Assuming that the first 30 points provide a good representation of the population mean and standard deviation, and that the data is normally distributed, the properties of the normal distribution tell us that just 0.26% of values in the population will fall outside of these $\pm3\sigma$ limits. This can be demonstrated using the `cdf()` and `sf()` of the normal distribution to calculate the area in each tail.

In [ ]:
std_x = np.linspace(-4, 4, 100)
std_norm = stats.norm.pdf(std_x, loc=0, scale=1)

# plot the pdf, cdf functions using previously calculated values
fig, ax = plt.subplots()

ax.plot(std_x, std_norm)

# Axes.axvspan(x_min, x_max) is used to create the color blocks at the tails
ax.axvspan(-4, -3, alpha=0.3)
ax.axvspan(3, 4, alpha=0.3)

ax.set_ylabel('Density')

# calculate the x value where cdf(x)=0.2
p_left_tail = stats.norm.cdf(-3, loc=0, scale=1)
p_right_tail = stats.norm.sf(3, loc=0, scale=1)

ax.text(-3.5, 0.3, f'{p_left_tail:0.2%}', ha='center')
ax.text(3.5, 0.3, f'{p_right_tail:0.2%}', ha='center')

When these $\pm3\sigma$ limits are applied to the $\bar{x}$ chart, it is clear that this chart indicates that the sample mean has shifted. All of the points, beginning with index 10 are out of control. The labels **UCL** and **LCL** refer to the upper and lower control limits. Sometimes the population mean is plotted as the **CL**, or center line.

In [ ]:
# calculate the mean and stdev based on the initial 30 points
mean_first30 = modulus[:30].mean()
std_first30 = modulus[:30].std(ddof=1)

# calculate the range for the control limits (6-sigma)
control_range = 6 * std_first30 / np.sqrt(3)

# plot the x-bar values
fig, ax = plt.subplots()

ax.plot(np.arange(len(x_bar)), x_bar, marker='x')

ax.set_xticks(np.arange(len(x_bar)))
ax.set_xlabel(r'Index of $\bar{x}$ Point')
ax.set_ylabel('Modulus (MPa)')
ax.set_title(r'$\bar{x}$ Chart')

# plot the mean and control limits
ax.axhline(mean_first30 + 0.5*control_range, ls='--')
ax.text(0, mean_first30 + 0.5*control_range + 20, "UCL")

ax.axhline(mean_first30, ls='--', color='gray', alpha=0.5)

ax.axhline(mean_first30 - 0.5*control_range, ls='--')
ax.text(0, mean_first30 - 0.5*control_range + 20, "LCL")

A common partner to the $\bar{x}$ chart is the $R$ chart, which displays the range ($x_{max}-x_{min}$) of each set of averaged values. If the range value is out of control, this indicates that variability amongst the samples is changing. The range is calculated based on the reshaped array, as was done for the mean.

In [ ]:
R = modulus_grouped.max(axis=1) - modulus_grouped.min(axis=1)
R

Control limits for the $R$ chart are a little different. You'll typically go to a table to get the values of several constants, which will depend on the number of measurements that are being average. Assuming you know (or have an estimate for) the standard deviation, the following formulas apply:

$$LCL=D_1\sigma$$
$$CL=d_1\sigma$$
$$UCL=D_2\sigma$$

The values can be found in a statistical quality control book such as [**Introduction to Statistical Quality Control**]() by Douglas Montgomery. In this example, we do not observe any out of control points in the $R$ chart that would indicate a change in variability over the course of the experiment. There is no evidence of a shift in variance from the R chart below.

In [ ]:
# calculate centerline (CL) and upper/lower control limits
R_CL = 1.693*std_first30
R_UCL = 4.358*std_first30
R_LCL = 0

# plot R chart
fig, ax = plt.subplots()

ax.plot(np.arange(len(R)), R, marker='x')

ax.set_xticks(np.arange(len(x_bar)))

ax.set_xlabel('Index of Data Point')
ax.set_ylabel('Modulus (MPa)')
ax.set_title(r'$R$ Chart')

# plot the centerline and control limits
ax.axhline(R_UCL, ls='--')
ax.text(0, R_UCL + 10, "UCL")

ax.axhline(R_CL, ls='--', color='gray', alpha=0.5)

ax.axhline(R_LCL, ls='--')
ax.text(0, R_LCL + 10, "LCL")

## 5.4. Testing 2 separate distributions

Based on what was observed in the $\bar{x}$ chart, we may also want to conduct a statistical test to determine if these two sets of data are from the same distribution. 

For clarity as we move through this exercise, the modulus array will be split into two pieces labels `mod_A` and `mod_B`, representing the first and last sets of 30 data points. This isn't necessary, but it will clean up the code and make it easier to understand.

In [ ]:
# split the modulus array into two separate arrays
# mod_A is the first 30 values, mod_B is the last 30
mod_A = modulus[:30]
mod_B = modulus[30:]

# plot the two distributions, with the overlaid normal distribution
fig, ax = plt.subplots() 

# plotting the first and second half of the data separately
# here, setting the transparency (alpha) to 40% helps visualize overlap
ax.hist(mod_A, bins=bin_edges, density=True, color='red', alpha=0.4, label='Modulus of A')
ax.hist(mod_B, bins=bin_edges, density=True, color='blue', alpha=0.4, label='Modulus of B')

# always remember to label the plots
ax.set_xlabel('Tensile Modulus (MPa)')
ax.set_ylabel('Frequency')
ax.legend()

# overlay the standard normal distributions for the two sets

# first, create an array of x-values to calculate the distributions over
x = np.linspace(modulus.min()-modulus.std(), modulus.max()+modulus.std(), 100)

# calculate the distributions, using the mean and stdev of the two sets
pdf_A = stats.norm.pdf(x, loc=mod_A.mean(), scale=mod_A.std(ddof=1))
pdf_B = stats.norm.pdf(x, loc=mod_B.mean(), scale=mod_B.std(ddof=1))

ax.plot(x, pdf_A, color='red')
ax.plot(x, pdf_B, color='blue')

These data sets visually appear to be different, but is there statistical evidence to support that assumption? This can be determined with the *t-test*. When applying a statistical hypothesis test, we must start with a *null hypothesis*, $H_0$, which will be that the mean of these two populations is the same: $\mu_A=\mu_B$. The *alternative hypothesis*, $H_1$, is that the population means are not the same: $\mu_A \ne \mu_B$. There are several versions of the *t-test* statistic, depending on whether or not the number of values is the same in both sample data sets ($n_A=n_B$) and whether the sample variances can be assumed as equal ($\sigma^2_A=\sigma^2_B$). 

For the assumption of equal variances, $\sigma^2_A=\sigma^2_B$, a pooled estimate of the variance is calculated from the two sample variances. 

$$s^2_p=\frac{\left(n_A-1\right)s^2_A + \left(n_B-1\right)s^2_B}{n_A+n_B-2}$$

Note the use of $s^2$ to indicate the *sample* standard deviation, which is again calculated based on $n-1$ degrees of freedom. To calculate this value using the `ndarray.var()` function, you must set the parameter `ddof=1`. Otherwise you'll calculate the *population* variance, based on $n$ degrees of freedom, and introduce a slight error.

The *t-statistic* for the null hypothesis is calculated as 

$$t_0=\frac{\bar{x}_A-\bar{x}_B}{s_p\sqrt{\frac{1}{n_A}-\frac{1}{n_B}}}$$

In [ ]:
# get the lengths of the arrays (could also use len(ndarray))
n_A = mod_A.size
n_B = mod_B.size

# calculate the pooled standard deviation, using the formula given above
sp = np.sqrt(((n_A-1)*mod_A.var(ddof=1) + (n_B-1)*mod_B.var(ddof=1))/(n_A+n_B-2))

# calculate the t-statistic, using the formula given above
t0 = (mod_A.mean()-mod_B.mean())/(sp*np.sqrt(1/n_A+1/n_B))

t0

The t-statistic will follow the standard t distribution, with the degrees of freedom given by: $df=n_A+n_B-2$. Using this, we can calculate the probability of this observed result if the null hypothesis of $\mu_A=\mu_B$ is actually true. Because this is a two-sided test (the alternative hypothesis is that $\mu_A$ could be less than *or* greater than $\mu_B$), we add the probability from the left and right tails of the distribution.

In [ ]:
# calculate the standard t-distribution for n_A+n_B-2 degrees of freedom
ttest_x = np.linspace(-13, 13, 500)
std_t = stats.t.pdf(ttest_x, loc=0, scale=1, df=(n_A+n_B-2))

# plot the pdf, cdf functions using previously calculated values
fig, ax = plt.subplots()

ax.plot(ttest_x, std_t)

# plot the tails, which correspond to our calculated t-statistic
ax.axvspan(-13, -abs(t0), alpha=0.3)
ax.axvspan(abs(t0), 13, alpha=0.3)

ax.set_ylabel('Density')

# calculate the x value where cdf(x)=0.2
p_left_tail = stats.t.cdf(-abs(t0), loc=0, scale=1, df=(n_A+n_B-2))
p_right_tail = stats.t.sf(abs(t0), loc=0, scale=1, df=(n_A+n_B-2))

ax.set_title(f'Sum of the left and right tails: {p_left_tail+p_right_tail:0.2e}')

Of course, there is a `scipy.stats` function, `stats.ttest_ind()`, that takes care of these calculations. The parameter, `equal_var`, that can be set as `True` or `False` to assert your assumption about equal variances for the two sample sets. In addition to the t-statistic, which matches our previous calculation, this function provides the *p-value* for the result. This is the probability of observing these data sets, given the null hypothesis being true. When the p-value is low, less than 0.05 or 0.01, we reject the null hypothesis as being statistically unlikely. The low *p-value* of $3.5 \times 10^-{17}$ indicates that we would reject the null hypothesis that $\mu_A=\mu_B$ as a statistical improbability.

In [ ]:
stats.ttest_ind(mod_A, mod_B, equal_var=True)

## 5.5. Confidence intervals

Closely related to the t-test is the concept of a confidence interval. The confidence interval provides a range estimate for a property, like a population mean, based on a given probability level (such as 95%). 

### 5.5.1. Confidence interval of a population mean

Given a set of $n$ data points, sampled at random from a normal probability distribution, we can put bounds on what the true population mean is. When the variance is unknown, as is often the case with experimental results, the confidence interval is calculated using the t-distribution. A two-sided confidence interval can be calculated from the standard t-distribution, using:

$$\bar{x}-t_{\alpha/2}\frac{s}{\sqrt{n}} \le \mu \le \bar{x}+t_{\alpha/2}\frac{s}{\sqrt{n}}$$

Where $\alpha$ represents the combined probability for the left and right tails. For a 95% confidence interval, $\alpha=1-0.95=0.05$. The degrees of freedom for the t-distribution are given by $n-1$, where $n$ is the number of data points.

This code calculates the 95% confidence intervale for the mean modulus of the film "A", from the previous data.

In [ ]:
interval_min = mod_A.mean() - stats.t.isf(0.05/2, df=(mod_A.size-1)) * mod_A.std(ddof=1)/np.sqrt(mod_A.size)
interval_max = mod_A.mean() + stats.t.isf(0.05/2, df=(mod_A.size-1)) * mod_A.std(ddof=1)/np.sqrt(mod_A.size)

f'({interval_min:0.2f}, {interval_max:0.2f})'

The confidence interval is more conveniently calculated using the `interval()` function from `scipy.stats`. Note that the value supplied as the standard deviation is actually the $\frac{s}{\sqrt{n}}$ term from the previous formula. As the number of samples averaged together increases, the confidence interval for the mean narrows.

In [ ]:
stats.t.interval(0.95, loc=mod_A.mean(), scale=(mod_A.std(ddof=1)/np.sqrt(mod_A.size)), df=(mod_A.size-1))

### 5.5.2. Confidence interval for a difference between means

The direct analog to the two sample t-test that we completed in section 5.4 is a confidence interval on the difference between means. For the assumption of equal variances, the same pooled standard deviation (calculated previously) is used. 

$$\bar{x}_A-\bar{x}_B-t_{\alpha/2}s_p{\sqrt{\frac{1}{n_A}+\frac{1}{n_B}}} \le \mu_A-\mu_B \le \bar{x}_A-\bar{x}_B+t_{\alpha/2}s_p{\sqrt{\frac{1}{n_A}+\frac{1}{n_B}}}$$

If the confidence interval for $\bar{x}_A-\bar{x}_B$ includes 0, this means that $\bar{x}_A=\bar{x}_B$ is included in the confidence interval, and the null hypothesis in the t-test will not be rejected. However, a confidence interval that does not include 0 will equate to rejection of the null hypothesis. The confidence interval on $\mu_A-\mu_B$ is far from 0, with would again indicate that the null hypothesis is rejected. The confidence interval method is statistically equivalent to the t-test, even though the output is different, so both tests are not necessary in a real world problem.

In [ ]:
stats.t.interval(0.95, loc=(mod_A.mean()-mod_B.mean()), scale=(sp*np.sqrt(1/n_A+1/n_B)), df=(n_A+n_B-2))

--------------
## Next Steps:

1. Complete the [Unit 5 Problems](./unit05-problems.ipynb) to test your understanding
2. Advance to [Unit 6](../06-regression-classification/unit06-lesson.ipynb) when you're ready for the next step